# Supply Chain Data Engineer Project

## Overview
This project focuses on analyzing supply chain data using Python and SQL. It involves extracting data from a SQL database, performing data cleaning, and preparing the data for analysis. Key tasks include checking for null values, summarizing quantitative data, and identifying delayed deliveries.

## Project Structure
1. **Data Connection**: Establish connection to the SQL database.
2. **Data Extraction**: Extract relevant data for analysis.
3. **Data Cleaning**:
   - Check for null values.
   - Summarize quantitative columns.
   - Identify duplicates.
   - Calculate delayed days and create a `not_delivered` column.
4. **Data Analysis**: Analyze the cleaned data to derive insights.

## Tools Used
- Python (Pandas, SQLAlchemy)
- SQL Server


## 1. Data Connection

In [2]:
# Import necessary libraries
import pandas as pd
import pyodbc as odbc


In [3]:
# Establish connection to the SQL database
sql_conn = odbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                        'SERVER=DESKTOP-3B6G2FC\MSSQLSERVER01;'
                        'DATABASE=supply_chain_1;'
                        'UID=Khalid;'
                        'PWD=246248246;')

## Load data from the SQL database into a DataFrame

In [5]:
query = "SELECT * FROM fact_order_lines"
df = pd.read_sql(query, sql_conn)

df

C:\Users\PC\AppData\Local\Temp\ipykernel_26344\2050240598.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, sql_conn)


,order_id,customer_id,product_id,order_qty,delivered_qty,order_placement_date,agreed_delivery_date,actual_delivery_date
0,FAP410101302,789101,25891103,493,493,2022-04-08,2022-04-10,2022-04-10
1,FAP410101302,789101,25891203,374,374,2022-04-08,2022-04-10,2022-04-10
2,FAP410101302,789101,25891302,46,44,2022-04-08,2022-04-10,2022-04-10
3,FAP410101402,789101,25891101,311,311,2022-04-07,2022-04-10,2022-04-10
4,FAP410101402,789101,25891201,442,442,2022-04-07,2022-04-10,2022-04-10
...,...,...,...,...,...,...,...,...
57091,FMY59903502,789903,25891502,105,100,2022-05-07,2022-05-09,2022-05-09
57092,FMY59903503,789903,25891102,327,294,2022-05-08,2022-05-09,2022-05-09
57093,FMY59903503,789903,25891503,127,121,2022-05-08,2022-05-09,2022-05-09
57094,FMY59903601,789903,25891601,91,86,2022-05-08,2022-05-09,2022-05-12


## Check for null values in the DataFrame

In [11]:
null_values = df.isnull().sum()
print("Null values in each column:")
print(null_values)

Null values in each column:
order_id                0
customer_id             0
product_id              0
order_qty               0
delivered_qty           0
order_placement_date    0
agreed_delivery_date    0
actual_delivery_date    0
dtype: int64


##  Check for Duplicates


In [12]:
duplicate_rows = df.duplicated().sum()
print(f"\nNumber of duplicate rows in the dataset: {duplicate_rows}")


print("\nDuplicate rows:")
print(df[df.duplicated()])




Number of duplicate rows in the dataset: 0

Duplicate rows:
Empty DataFrame
Columns: [order_id, customer_id, product_id, order_qty, delivered_qty, order_placement_date, agreed_delivery_date, actual_delivery_date]
Index: []


0        FAP410101302
1        FAP410101302
2        FAP410101302
3        FAP410101402
4        FAP410101402
             ...     
57091     FMY59903502
57092     FMY59903503
57093     FMY59903503
57094     FMY59903601
57095     FMY59903603
Name: order_id, Length: 57096, dtype: object

##  Add a new column for the number of delayed days


In [15]:
df['order_placement_date'] = pd.to_datetime(df['order_placement_date'])
df['agreed_delivery_date'] = pd.to_datetime(df['agreed_delivery_date'])
df['actual_delivery_date'] = pd.to_datetime(df['actual_delivery_date'])


df['delay_days'] = (df['actual_delivery_date'] - df['agreed_delivery_date']).dt.days




In [16]:
print("\nSample of the data with delay days:")
print(df.tail())


Sample of the data with delay days:
          order_id  customer_id  product_id  order_qty  delivered_qty  \
57091  FMY59903502       789903    25891502        105            100   
57092  FMY59903503       789903    25891102        327            294   
57093  FMY59903503       789903    25891503        127            121   
57094  FMY59903601       789903    25891601         91             86   
57095  FMY59903603       789903    25891603         96             91   

      order_placement_date agreed_delivery_date actual_delivery_date  \
57091           2022-05-07           2022-05-09           2022-05-09   
57092           2022-05-08           2022-05-09           2022-05-09   
57093           2022-05-08           2022-05-09           2022-05-09   
57094           2022-05-08           2022-05-09           2022-05-12   
57095           2022-05-06           2022-05-09           2022-05-10   

       delay_days  
57091           0  
57092           0  
57093           0  
57094      

## Calculate not delivered quantity by subtracting delivered_qty from order_qty


In [22]:
df['not_delivered'] = (df['order_qty'] - df['delivered_qty']).astype(int)



# Show specific columns (order_id, customer_id, product_id, delay_days, not_delivered)
print("\nSample of selected columns with delay days and not delivered quantity:")
print(df[['order_id','product_id', 'order_qty', 'delivered_qty', 'not_delivered']].head())


Sample of selected columns with delay days and not delivered quantity:
       order_id  product_id  order_qty  delivered_qty  not_delivered
0  FAP410101302    25891103        493            493              0
1  FAP410101302    25891203        374            374              0
2  FAP410101302    25891302         46             44              2
3  FAP410101402    25891101        311            311              0
4  FAP410101402    25891201        442            442              0


In [23]:
# Describe the 'not_delivered' column to get statistical summary
not_delivered_summary = df['not_delivered'].describe()


# Print the summary
print("\nStatistical summary of not delivered quantities:")
print(not_delivered_summary)


Statistical summary of not delivered quantities:
count    57096.000000
mean         8.017707
std         16.484549
min          0.000000
25%          0.000000
50%          0.000000
75%          9.000000
max        100.000000
Name: not_delivered, dtype: float64
